In [1]:
import os, math, collections, itertools

from tqdm import tqdm

import matplotlib.pyplot as plt
import tensorflow as tf
import skimage.measure
import numpy as np

from utils import save, unpickle, pixel_range, preprocess, batch_run, scale
from deepexplain.tensorflow import DeepExplain
from classifiers import CIFAR_CNN
from trainers import Trainer
from attacks import GM

os.environ["CUDA_VISIBLE_DEVICES"]="3"

datadir = './CIFAR10_data/'
batches = [datadir + batch for batch in os.listdir(datadir)]

for i in tqdm(range(5)):
    batch = unpickle(batches[i])

    if i == 0:
        data = batch[b'data'].astype(np.float32)
        cifar = np.transpose(np.reshape(data, [-1,3,32,32]), [0,2,3,1])
        labels = batch[b'labels']
    else:
        data = batch[b'data'].astype(np.float32)
        cifar = np.concatenate((cifar, np.transpose(np.reshape(data, [-1,3,32,32]), [0,2,3,1])), axis=0)
        labels = np.concatenate((labels, batch[b'labels']), axis=0)

scaled_cifar = cifar / 127.5 - 1.0

test_batch = unpickle(batches[5])
cifar_test = np.transpose(np.reshape(test_batch[b'data'], [-1,3,32,32]), [0,2,3,1])
scaled_cifar_test = cifar_test / 127.5 - 1.0
labels_test = np.array(test_batch[b'labels'])

data_train = (scaled_cifar, labels)
data_test = (scaled_cifar_test, labels_test)

cifar_mean = np.mean(cifar, axis=(0,1,2))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
100%|██████████| 5/5 [00:02<00:00,  1.86it/s]


## Accuracy

In [ ]:
def eval_accuracy(logdir):
    
    tf.reset_default_graph()
    
    sess = tf.InteractiveSession()
    
    DNN = CIFAR_CNN(logdir, tf.nn.relu)
    DNN.load(sess)
    
    gm = GM(DNN, eps=0.4, step_size=2, n_steps=40, norm='2', loss_type='xent')
    
    train_acc = DNN.evaluate(sess, data_train)
    test_acc = DNN.evaluate(sess, data_test)
    adv_acc = DNN.evaluate(sess, (gm.attack(sess, data_test, batch_size=500), data_test[1]))
    
    sess.close()
    
    res = collections.OrderedDict([('Train', train_acc), ('Test', test_acc), ('Adv', adv_acc)])
    
    savefile = logdir.replace('tf_logs', 'results/accuracy')[:-1] + '.pickle'
    savedir = '/'.join(savefile.split('/')[:-1])
    
    if not os.path.exists(savedir):
        
        os.makedirs(savedir)
    
    save(res, savefile)

## ROAR KAR

In [3]:
def nonzero_min(x):
    
    return np.min(np.abs(x[x != 0]))

def Canny(images):
    
    res = []
    
    for img in images:
        
        img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.uint8)
        edges = cv2.Canny(img_bw, 100, 200).reshape(1, 32, 32) + np.random.normal(loc=0.0, scale=0.1, size=[1, 32, 32])
        res.append(edges)
    
    return np.concatenate(res, axis=0)


def random_remove(images, percentile, keep=False):
    
    images = np.copy(images)
    
    mask = np.random.binomial(1, (100 - percentile) / 100, size=images.shape[:-1])
    
    if keep:
        images[mask == 1] = cifar_mean
    else:
        images[mask == 0] = cifar_mean
    
    return images


def remove(images, attributions, percentile, keep=False, random=False):
    '''
    images       : tensor of shape [N,H,W,C]
    attributions : tensor of shape [N,H,W]
    percentile   : scalar between 0 and 100, inclusive
    keep         : if true keep q percent; otherwise remove q percent
    '''
    
    images = np.copy(images)
    
    thresholds = np.percentile(attributions, 100 - percentile, axis=(1,2), keepdims=True)
    
    if keep:
        images[attributions < thresholds] = cifar_mean
    else:
        images[attributions > thresholds] = cifar_mean
    
    return images


def occlude_dataset(DNN, de, attribution, loss, percentiles, test=False, keep=False, random=False, batch_size=1000, savedir='./'):
    
    if test:
        Xs = cifar_test
        ys = labels_test
    else:
        Xs = cifar
        ys = labels
    
    total_batch = math.ceil(len(Xs) / batch_size)
    
    if not random:
    
        hmaps = []

        for i in tqdm(range(total_batch)):

            batch_xs = Xs[i*batch_size:(i+1)*batch_size]
            batch_xs_scaled = scale(batch_xs)

            if 'edge' in attribution:
                attrs = Canny(batch_xs)
            else:
                attrs = preprocess(de.explain(attribution, loss, DNN.X, batch_xs_scaled), 0, 100, use_abs=True)
            
            # Add small random noise so np.percentile works correctly
            attrs += np.random.uniform(low=0.0, high=1e-8, size=attrs.shape)
            
            hmaps.append(attrs)
        
        hmaps = np.concatenate(hmaps, axis=0)
    
    for percentile in tqdm(percentiles):
        
        dataset = []
        
        for i in range(total_batch):
            
            batch_xs, batch_ys = Xs[i*batch_size:(i+1)*batch_size], ys[i*batch_size:(i+1)*batch_size]
            
            if random:
                occluded_images = random_remove(batch_xs, percentile, keep)
            else:
                batch_attrs = hmaps[i * batch_size:(i + 1) * batch_size]
                occluded_images = remove(batch_xs, batch_attrs, percentile, keep)
            
            dataset.append(scale(occluded_images))
        
        save(np.concatenate(dataset, axis=0), savedir + '{}_{}_{}.pickle'.format('test' if test else 'train', attribution, percentile))


def eval_roar_kar(logdir, keep, loss_type='xent', train_only=False):
    
    def get_savedir():
        
        savedir = logdir.replace('tf_logs', 'KAR_' + loss_type.upper() if keep else 'ROAR_' + loss_type.upper())
        
        if not os.path.exists(savedir):
            
            os.makedirs(savedir)
        
        return savedir
    
    percentiles = [10, 30, 50, 70, 90]
    
    heavy_methods = ['SmoothGrad', 'IntegGrad', 'DeepLIFT']

    attribution_methods = [
#                            ('Random'           , 'zero'), 
#                            ('Saliency'         , 'saliency'), 
#                            ('Guided BP'        , 'guidedbp'), 
#                            ('SmoothGrad'       , 'smoothgrad'), 
                           ('Grad * Input'     , 'grad*input'), 
#                            ('IntegGrad'        , 'intgrad'), 
#                            ('DeepLIFT'         , 'deeplift')
                          ]

    attribution_methods = collections.OrderedDict(attribution_methods)
    
    if not train_only:
    
        tf.reset_default_graph()

        sess = tf.InteractiveSession()

        with DeepExplain(session=sess, graph=sess.graph) as de:

            DNN = CIFAR_CNN(logdir, activation=tf.nn.relu)
            DNN.load(sess)
            
            if loss_type == 'xent':
                loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes)), logits=DNN.logits)
            elif loss_type == 'cw':
                label_mask = tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes, on_value=True, off_value=False))
                c_logit = tf.boolean_mask(DNN.logits, label_mask)
                w_logit = tf.reduce_max(tf.reshape(tf.boolean_mask(DNN.logits, tf.logical_not(label_mask)), shape=[-1,9]), axis=1)
                loss = w_logit - c_logit
            else:
                raise Exception('Unknown loss type')

            for k, v in attribution_methods.items():

                if k in heavy_methods:
                    batch_size = 2500
                else:
                    batch_size = 2500
                
                occlude_dataset(DNN, de, v, loss, percentiles, False, keep, k == 'Random', batch_size, get_savedir())
                occlude_dataset(DNN, de, v, loss, percentiles, True,  keep, k == 'Random', batch_size, get_savedir())

        sess.close()
    
    ress = collections.OrderedDict([(k, []) for k in attribution_methods.keys()])
    
    for _ in range(3):
        
        for k, v in attribution_methods.items():
            
            res = []

            for p in percentiles:
                
                occdir = get_savedir() + '{}_{}_{}.pickle'.format('{}', v, p)
                data_train = (unpickle(occdir.format('train')), labels)
                data_test = (unpickle(occdir.format('test')), labels_test)
                
                tf.reset_default_graph()
                
                DNN = CIFAR_CNN('tf_logs/exp3/KAR_Loss/', activation=tf.nn.relu)
                
                sess = tf.InteractiveSession()
                sess.run(tf.global_variables_initializer())
                
                trainer = Trainer(sess, DNN, data_train)
                trainer.train(20, p_epochs=30)
                
                acc = DNN.evaluate(sess, data_test)
                
                print('{}{} | Accuracy : {:.5f}'.format(k, p, acc))
                
                res.append(acc)
                
                sess.close()
                
            ress[k].append(res)
    
    res_mean = {k: np.mean(v, axis=0) for k, v in ress.items()}
    
    savefile = logdir.replace('tf_logs', 'results/{}'.format('kar_' + loss_type if keep else 'roar_' + loss_type))[:-1] + '.pickle'
    savedir = '/'.join(savefile.split('/')[:-1])
    
    print('Saving results at ' + savefile)
    
    if not os.path.exists(savedir):
        
        os.makedirs(savedir)
    
    if not os.path.isfile(savefile):
    
        save(res_mean, savefile)
    
    else:
        
        res_prev = unpickle(savefile)
        
        for k, v in res_mean.items():
        
            res_prev[k] = v
        
        save(res_prev, savefile)

## Train

In [4]:
def params_maker(param_names, param_values):
    
    product = itertools.product(*param_values)
    params = [list(zip(param_names, p)) for p in product]
    
    return params

def logdir_maker(attack_params):
    
    return 'tf_logs/exp3/adv/GM/{}/'.format('_'.join([str(v) for k, v in attack_params]))

def run(attack_params=None, train_first=True, run_sensitivity=True, run_roar_kar=True, loss_type='xent', keep=False, rk_train_only=False):
    
    if attack_params:
        logdir = logdir_maker(attack_params)
    else:
        logdir = 'tf_logs/exp3/standard/'
    
    print(logdir)
    
    if train_first:
    
        tf.reset_default_graph()

        DNN = CIFAR_CNN(logdir, activation=tf.nn.relu, attack=GM, attack_params=dict(attack_params))

        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())

        trainer = Trainer(sess, DNN, data_train)
        trainer.train(n_epochs=20, p_epochs=5)

        sess.close()
        
        eval_accuracy(logdir)
    
    if run_roar_kar: eval_roar_kar(logdir, keep, loss_type, rk_train_only)

In [5]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.002 * i, 3) for i in range(1,21)], [0.02], [40], ['inf'], ['xent']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=False)

tf_logs/exp4/adv/GM/0.002_0.02_40_inf_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.002_0.02_40_inf_xent/model


100%|██████████| 5/5 [00:37<00:00,  7.54s/it]


Grad * Input10 | Accuracy : 0.65370
Grad * Input30 | Accuracy : 0.60390
Grad * Input50 | Accuracy : 0.55920
Grad * Input70 | Accuracy : 0.51870
Grad * Input90 | Accuracy : 0.43830
Grad * Input10 | Accuracy : 0.64670
Grad * Input30 | Accuracy : 0.60330
Grad * Input50 | Accuracy : 0.55050
Grad * Input70 | Accuracy : 0.52160
Grad * Input90 | Accuracy : 0.43810
Grad * Input10 | Accuracy : 0.64300
Grad * Input30 | Accuracy : 0.60660
Grad * Input50 | Accuracy : 0.56480
Grad * Input70 | Accuracy : 0.50560
Grad * Input90 | Accuracy : 0.42970
Saving results at results/roar_xent/exp4/adv/GM/0.002_0.02_40_inf_xent.pickle
tf_logs/exp4/adv/GM/0.004_0.02_40_inf_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.004_0.02_40_inf_xent/model


100%|██████████| 5/5 [00:32<00:00,  6.50s/it]


Grad * Input10 | Accuracy : 0.65830
Grad * Input30 | Accuracy : 0.59750
Grad * Input50 | Accuracy : 0.55900
Grad * Input70 | Accuracy : 0.50660
Grad * Input90 | Accuracy : 0.42890
Grad * Input10 | Accuracy : 0.64580
Grad * Input30 | Accuracy : 0.59470
Grad * Input50 | Accuracy : 0.56020
Grad * Input70 | Accuracy : 0.50610
Grad * Input90 | Accuracy : 0.43260
Grad * Input10 | Accuracy : 0.64450
Grad * Input30 | Accuracy : 0.59070
Grad * Input50 | Accuracy : 0.56290
Grad * Input70 | Accuracy : 0.49310
Grad * Input90 | Accuracy : 0.42690
Saving results at results/roar_xent/exp4/adv/GM/0.004_0.02_40_inf_xent.pickle
tf_logs/exp4/adv/GM/0.006_0.02_40_inf_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.006_0.02_40_inf_xent/model


...

Grad * Input10 | Accuracy : 0.65210
Grad * Input30 | Accuracy : 0.57680
Grad * Input50 | Accuracy : 0.51750
Grad * Input70 | Accuracy : 0.46950
Grad * Input90 | Accuracy : 0.37880
Grad * Input10 | Accuracy : 0.65320
Grad * Input30 | Accuracy : 0.56580
Grad * Input50 | Accuracy : 0.50530
Grad * Input70 | Accuracy : 0.47110
Grad * Input90 | Accuracy : 0.37660
Grad * Input10 | Accuracy : 0.65780
Grad * Input30 | Accuracy : 0.57210
Grad * Input50 | Accuracy : 0.51030
Grad * Input70 | Accuracy : 0.47780
Grad * Input90 | Accuracy : 0.38150
Saving results at results/roar_xent/exp4/adv/GM/0.036_0.02_40_inf_xent.pickle
tf_logs/exp4/adv/GM/0.038_0.02_40_inf_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.038_0.02_40_inf_xent/model


100%|██████████| 5/5 [00:46<00:00,  9.24s/it]


Grad * Input10 | Accuracy : 0.65070
Grad * Input30 | Accuracy : 0.56080
Grad * Input50 | Accuracy : 0.50770
Grad * Input70 | Accuracy : 0.45740
Grad * Input90 | Accuracy : 0.38450
Grad * Input10 | Accuracy : 0.65840
Grad * Input30 | Accuracy : 0.56680
Grad * Input50 | Accuracy : 0.51020
Grad * Input70 | Accuracy : 0.46520
Grad * Input90 | Accuracy : 0.37940
Grad * Input10 | Accuracy : 0.65290
Grad * Input30 | Accuracy : 0.56250
Grad * Input50 | Accuracy : 0.51160
Grad * Input70 | Accuracy : 0.46400
Grad * Input90 | Accuracy : 0.37070
Saving results at results/roar_xent/exp4/adv/GM/0.038_0.02_40_inf_xent.pickle
tf_logs/exp4/adv/GM/0.04_0.02_40_inf_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.04_0.02_40_inf_xent/model


100%|██████████| 5/5 [00:47<00:00,  9.56s/it]


Grad * Input10 | Accuracy : 0.63760
Grad * Input30 | Accuracy : 0.55940
Grad * Input50 | Accuracy : 0.50240
Grad * Input70 | Accuracy : 0.45610
Grad * Input90 | Accuracy : 0.37860
Grad * Input10 | Accuracy : 0.64810
Grad * Input30 | Accuracy : 0.55830
Grad * Input50 | Accuracy : 0.52580
Grad * Input70 | Accuracy : 0.47290
Grad * Input90 | Accuracy : 0.39170
Grad * Input10 | Accuracy : 0.64330
Grad * Input30 | Accuracy : 0.56700
Grad * Input50 | Accuracy : 0.51920
Grad * Input70 | Accuracy : 0.45130
Grad * Input90 | Accuracy : 0.38100
Saving results at results/roar_xent/exp4/adv/GM/0.04_0.02_40_inf_xent.pickle


In [5]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.002 * i, 3) for i in range(8,21)], [0.02], [40], ['inf'], ['cw']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=False)

tf_logs/exp4/adv/GM/0.016_0.02_40_inf_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.016_0.02_40_inf_cw/model


100%|██████████| 5/5 [00:18<00:00,  3.69s/it]


Grad * Input10 | Accuracy : 0.64600
Grad * Input30 | Accuracy : 0.57830
Grad * Input50 | Accuracy : 0.52400
Grad * Input70 | Accuracy : 0.48660
Grad * Input90 | Accuracy : 0.41270
Grad * Input10 | Accuracy : 0.64860
Grad * Input30 | Accuracy : 0.58140
Grad * Input50 | Accuracy : 0.53560
Grad * Input70 | Accuracy : 0.47900
Grad * Input90 | Accuracy : 0.39960
Grad * Input10 | Accuracy : 0.63110
Grad * Input30 | Accuracy : 0.57610
Grad * Input50 | Accuracy : 0.53080
Grad * Input70 | Accuracy : 0.48080
Grad * Input90 | Accuracy : 0.40710
Saving results at results/roar_xent/exp4/adv/GM/0.016_0.02_40_inf_cw.pickle
tf_logs/exp4/adv/GM/0.018_0.02_40_inf_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.018_0.02_40_inf_cw/model


100%|██████████| 5/5 [00:25<00:00,  5.17s/it]


Grad * Input10 | Accuracy : 0.64960
Grad * Input30 | Accuracy : 0.58380
Grad * Input50 | Accuracy : 0.54030
Grad * Input70 | Accuracy : 0.47650
Grad * Input90 | Accuracy : 0.40660
Grad * Input10 | Accuracy : 0.64680
Grad * Input30 | Accuracy : 0.58670
Grad * Input50 | Accuracy : 0.52540
Grad * Input70 | Accuracy : 0.47240
Grad * Input90 | Accuracy : 0.40360
Grad * Input10 | Accuracy : 0.65500
Grad * Input30 | Accuracy : 0.57010
Grad * Input50 | Accuracy : 0.52390
Grad * Input70 | Accuracy : 0.47530
Grad * Input90 | Accuracy : 0.39970
Saving results at results/roar_xent/exp4/adv/GM/0.018_0.02_40_inf_cw.pickle
tf_logs/exp4/adv/GM/0.02_0.02_40_inf_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.02_0.02_40_inf_cw/model


...

Grad * Input10 | Accuracy : 0.65070
Grad * Input30 | Accuracy : 0.56560
Grad * Input50 | Accuracy : 0.51040
Grad * Input70 | Accuracy : 0.46550
Grad * Input90 | Accuracy : 0.38610
Grad * Input10 | Accuracy : 0.64500
Grad * Input30 | Accuracy : 0.55720
Grad * Input50 | Accuracy : 0.51010
Grad * Input70 | Accuracy : 0.45600
Grad * Input90 | Accuracy : 0.38700
Grad * Input10 | Accuracy : 0.64290
Grad * Input30 | Accuracy : 0.56520
Grad * Input50 | Accuracy : 0.49950
Grad * Input70 | Accuracy : 0.44520
Grad * Input90 | Accuracy : 0.38660
Saving results at results/roar_xent/exp4/adv/GM/0.036_0.02_40_inf_cw.pickle
tf_logs/exp4/adv/GM/0.038_0.02_40_inf_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.038_0.02_40_inf_cw/model


100%|██████████| 5/5 [00:47<00:00,  9.45s/it]


Grad * Input10 | Accuracy : 0.65940
Grad * Input30 | Accuracy : 0.57260
Grad * Input50 | Accuracy : 0.51300
Grad * Input70 | Accuracy : 0.46000
Grad * Input90 | Accuracy : 0.38790
Grad * Input10 | Accuracy : 0.65590
Grad * Input30 | Accuracy : 0.56560
Grad * Input50 | Accuracy : 0.50450
Grad * Input70 | Accuracy : 0.46180
Grad * Input90 | Accuracy : 0.37680
Grad * Input10 | Accuracy : 0.64930
Grad * Input30 | Accuracy : 0.56410
Grad * Input50 | Accuracy : 0.50420
Grad * Input70 | Accuracy : 0.45150
Grad * Input90 | Accuracy : 0.38210
Saving results at results/roar_xent/exp4/adv/GM/0.038_0.02_40_inf_cw.pickle
tf_logs/exp4/adv/GM/0.04_0.02_40_inf_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.04_0.02_40_inf_cw/model


100%|██████████| 5/5 [00:35<00:00,  7.10s/it]


Grad * Input10 | Accuracy : 0.64820
Grad * Input30 | Accuracy : 0.56600
Grad * Input50 | Accuracy : 0.51120
Grad * Input70 | Accuracy : 0.46110
Grad * Input90 | Accuracy : 0.38250
Grad * Input10 | Accuracy : 0.64890
Grad * Input30 | Accuracy : 0.56210
Grad * Input50 | Accuracy : 0.50580
Grad * Input70 | Accuracy : 0.45580
Grad * Input90 | Accuracy : 0.38080
Grad * Input10 | Accuracy : 0.64560
Grad * Input30 | Accuracy : 0.55380
Grad * Input50 | Accuracy : 0.50150
Grad * Input70 | Accuracy : 0.45620
Grad * Input90 | Accuracy : 0.38390
Saving results at results/roar_xent/exp4/adv/GM/0.04_0.02_40_inf_cw.pickle


In [ ]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.002 * i, 3) for i in range(1,21)], [0.02], [40], ['inf'], ['xent']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=True)

In [ ]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.002 * i, 3) for i in range(1,21)], [0.02], [40], ['inf'], ['cw']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=True)